# OLID datasets

## import libraries

In [1]:
import pandas as pd
import string
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split


## load the datasets

In [3]:
train_data=pd.read_csv('./olid-training-v1.0.tsv', delimiter='\t', encoding='utf-8')

train_tweets = train_data[['tweet']] #Extract tweets
train_task_a_labels= train_data[['subtask_a']] #Extract subtsak_a labels

train_task_a_labels.columns.values[0] = 'class_a' #Rename class attribute


print(train_data)

          id                                              tweet subtask_a  \
0      86426  @USER She should ask a few native Americans wh...       OFF   
1      90194  @USER @USER Go home you’re drunk!!! @USER #MAG...       OFF   
2      16820  Amazon is investigating Chinese employees who ...       NOT   
3      62688  @USER Someone should'veTaken" this piece of sh...       OFF   
4      43605  @USER @USER Obama wanted liberals &amp; illega...       NOT   
...      ...                                                ...       ...   
13235  95338  @USER Sometimes I get strong vibes from people...       OFF   
13236  67210  Benidorm ✅  Creamfields ✅  Maga ✅   Not too sh...       NOT   
13237  82921  @USER And why report this garbage.  We don't g...       OFF   
13238  27429                                        @USER Pussy       OFF   
13239  46552  #Spanishrevenge vs. #justice #HumanRights and ...       NOT   

      subtask_b subtask_c  
0           UNT       NaN  
1           TIN    

In [4]:
train_data

,id,tweet,subtask_a,subtask_b,subtask_c
0,86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND
2,16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN
3,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN
4,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN
...,...,...,...,...,...
13235,95338,@USER Sometimes I get strong vibes from people...,OFF,TIN,IND
13236,67210,Benidorm ✅ Creamfields ✅ Maga ✅ Not too sh...,NOT,NaN,NaN
13237,82921,@USER And why report this garbage. We don't g...,OFF,TIN,OTH
13238,27429,@USER Pussy,OFF,UNT,NaN


In [5]:
train_data['subtask_a'].value_counts()

NOT    8840
OFF    4400
Name: subtask_a, dtype: int64

## NLP

In [6]:
!pip install emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 5.1 MB/s eta 0:00:00


In [7]:
import emoji
import re

In [8]:
def replace_multiple_user_mentions(text):
    """
    Replaces multiple '@USER' mentions in a tweet with a single '@USERS'.
    """
    user_mentions = len(re.findall('@USER', text))
    if user_mentions > 1:
        text = re.sub('@USER', '@USERS', text)
    return text


In [9]:

def emoji_to_text(text):
    return emoji.demojize(text, delimiters=("", ""))

In [10]:
def clean_tweets(df):
    punctuations = string.punctuation

    df.loc[:, 'tweet'] = df.tweet.str.replace('@USER', '') #Remove mentions (@USER)
    df.loc[:, 'tweet'] = df.tweet.str.replace('URL', '') #Remove URLs
    df.loc[:, 'tweet'] = df.tweet.str.replace('&amp', 'and') #Replace ampersand (&) with and
    df.loc[:, 'tweet'] = df.tweet.str.replace('&lt','') #Remove &lt
    df.loc[:, 'tweet'] = df.tweet.str.replace('&gt','') #Remove &gt
    df.loc[:, 'tweet'] = df.tweet.str.replace('\d+','') #Remove numbers
    df.loc[:, 'tweet'] = df.tweet.str.lower() #Lowercase

    #Remove punctuations
    for punctuation in punctuations:
        df.loc[:, 'tweet'] = df.tweet.str.replace(punctuation, '')

        df['tweet'] = df['tweet'].apply(
        lambda x: x.encode('ascii', 'ignore').decode('ascii')
)

    # Transfer emojis to segmentations
    df.loc[:, 'tweet'] = df['tweet'].apply(emoji.demojize)
    #Remove emojis
    #df.loc[:, 'tweet'] = df.tweet.str.strip() #Trim leading and trailing whitespaces
    # Replace multiple user mentions
    df.loc[:, 'tweet'] = df['tweet'].apply(replace_multiple_user_mentions)

    # Substitute out-of-vocabulary (OOV) words
    df.loc[:, 'tweet'] = df['tweet'].apply(lambda x: re.sub(r'http\S+|www\S+', 'http', x))


In [11]:
clean_tweets(train_tweets)

<ipython-input-10-58d0d99cd0a4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'tweet'] = df.tweet.str.replace('@USER', '') #Remove mentions (@USER)
<ipython-input-10-58d0d99cd0a4>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'tweet'] = df.tweet.str.replace('URL', '') #Remove URLs
<ipython-input-10-58d0d99cd0a4>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [12]:
train_task_a_data = train_tweets.join(train_task_a_labels)
train_task_a_data.update(train_task_a_data[['tweet']].applymap('\'{}\''.format))


In [13]:
train_task_a_data.head()

,tweet,class_a
0,' she should ask a few native americans what t...,OFF
1,' go home youre drunk maga trump ',OFF
2,'amazon is investigating chinese employees who...,NOT
3,' someone shouldvetaken this piece of shit to ...,OFF
4,' obama wanted liberals and illegals to move ...,NOT


In [14]:
#Read tweets from test sets
test_tweet_a=pd.read_csv('./testset-levela.tsv', delimiter='\t', encoding='utf-8')

#Read tweet labels
test_label_a=pd.read_csv('./labels-levela.csv', encoding='utf-8',
                         index_col=False, names=['id', 'class_a'])


#Merge tweets with labels by id
test_tweet_a = test_tweet_a.merge(test_label_a, on='id')

#Drop id column
test_tweet_a = test_tweet_a.drop(columns='id')

#Clean tweets in test sets
clean_tweets(test_tweet_a)

#Apply quotes to cleaned tweets
test_tweet_a.update(test_tweet_a[['tweet']].applymap('\'{}\''.format))


test_tweet_a.head()

<ipython-input-10-58d0d99cd0a4>:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, 'tweet'] = df.tweet.str.replace('\d+','') #Remove numbers
<ipython-input-10-58d0d99cd0a4>:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.loc[:, 'tweet'] = df.tweet.str.replace(punctuation, '')


,tweet,class_a
0,'whoisq wherestheserver dumpnike declasfisa de...,OFF
1,'constitutionday is revered by conservatives h...,NOT
2,'foxnews nra maga potus trump ndamendment rnc ...,NOT
3,'watching boomer getting the news that she is ...,NOT
4,'nopasaran unity demo to oppose the farright i...,OFF


# Task A

In [15]:
# Creating labelEncoder
encoder = LabelEncoder()
# Converting string labels into numbers.
train_task_a_data["class_a_code"] = encoder.fit_transform(train_task_a_data["class_a"])
test_tweet_a["class_a_code"] = encoder.fit_transform(test_tweet_a["class_a"])

In [16]:
#Create tuple pair for class and class code
train_task_a_data['class-tuple'] = train_task_a_data[['class_a', 'class_a_code']].apply(tuple, axis=1)
class_a = train_task_a_data['class-tuple'].unique()
class_a

array([('OFF', 1), ('NOT', 0)], dtype=object)

In [17]:
# Label the Data

train_tweets = train_task_a_data['tweet']
train_labels = train_task_a_data['class_a_code']

X_test = test_tweet_a['tweet']
y_test = test_tweet_a['class_a_code']


print("Shape of train_tweet is {} and shape of train_label is {}".format(train_tweets.shape, train_labels.shape))
print("Shape of test_tweet is {} and shape of test_label is {}".format(X_test.shape, y_test.shape))

Shape of train_tweet is (13240,) and shape of train_label is (13240,)
Shape of test_tweet is (860,) and shape of test_label is (860,)


In [18]:
X_train,X_val,y_train,y_val = train_test_split(train_tweets,train_labels,test_size=0.30,random_state=42)


print("Shape of test_tweets is {} and shape of test_labels is {}".format(X_train.shape, y_train.shape))

Shape of test_tweets is (9268,) and shape of test_labels is (9268,)


In [19]:
X_train

2804               '  truly anything to avoid gun control'
13084    ' its not wrongfully characterizing its the fu...
2211     '  they need to add more gun control laws and ...
11154    'boom  maybe that book deal isnt such a good i...
10620    ' unfirtunately i have seen hispanics justifyi...
                               ...                        
11964    ' isnt there a statute of limitations on sexua...
5191             '  he is probably dying due to dog fumes'
5390                                  '  you are so right'
860           ' sams act is working i swear she is acting'
7270                                     '  chuckle heads'
Name: tweet, Length: 9268, dtype: object

In [20]:
y_train

2804     0
13084    1
2211     0
11154    0
10620    1
        ..
11964    0
5191     0
5390     0
860      0
7270     0
Name: class_a_code, Length: 9268, dtype: int64

In [21]:
test_tweet_a

,tweet,class_a,class_a_code
0,'whoisq wherestheserver dumpnike declasfisa de...,OFF,1
1,'constitutionday is revered by conservatives h...,NOT,0
2,'foxnews nra maga potus trump ndamendment rnc ...,NOT,0
3,'watching boomer getting the news that she is ...,NOT,0
4,'nopasaran unity demo to oppose the farright i...,OFF,1
...,...,...,...
855,'despicabledems lie again about rifles dem dis...,OFF,1
856,'meetthespeakers will present in our event o...,NOT,0
857,' people just unfollowed me for talking about ...,OFF,1
858,'wednesdaywisdom antifa calls the right fascis...,NOT,0


In [22]:
X_test

0      'whoisq wherestheserver dumpnike declasfisa de...
1      'constitutionday is revered by conservatives h...
2      'foxnews nra maga potus trump ndamendment rnc ...
3      'watching boomer getting the news that she is ...
4      'nopasaran unity demo to oppose the farright i...
                             ...                        
855    'despicabledems lie again about rifles dem dis...
856    'meetthespeakers   will present in our event o...
857    ' people just unfollowed me for talking about ...
858    'wednesdaywisdom antifa calls the right fascis...
859             'kavanaugh typical liberals  democrats '
Name: tweet, Length: 860, dtype: object

In [23]:
y_test

0      1
1      0
2      0
3      0
4      1
      ..
855    1
856    0
857    1
858    0
859    0
Name: class_a_code, Length: 860, dtype: int64

In [28]:
!pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.9 MB/s eta 0:00:00


In [30]:
from transformers import BertTokenizerFast, TFBertForSequenceClassification
from transformers import glue_convert_examples_to_features

import tensorflow as tf

# Use the same tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

# Encode Labels
encoder = LabelEncoder()
train_labels = encoder.fit_transform(y_train)
val_labels = encoder.transform(y_val)
test_labels = encoder.transform(y_test)

# Tokenize and align labels with the tokenized inputs
train_encodings = tokenizer(list(X_train), truncation=True, padding=True, max_length=256)
val_encodings = tokenizer(list(X_val), truncation=True, padding=True, max_length=256)
test_encodings = tokenizer(list(X_test), truncation=True, padding=True, max_length=256)



In [31]:
# Convert to TensorFlow Datasets
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
)).shuffle(1000).batch(16)

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
)).batch(16)

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    test_labels
)).batch(16)

In [32]:

# Initialize the BERT model (make sure the number of labels match your dataset)
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(class_a))

# Choose an optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-6)

# Choose a loss function
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Compile the model
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [33]:
from tensorflow.keras.callbacks import EarlyStopping


# Define early stopping
early_stopping = EarlyStopping(
    monitor='val_loss', # usually val_loss or val_acc
    mode='min', # the direction that you want to monitor (min for loss, max for accuracy)
    patience=3, # number of epochs to wait before stopping the training
    verbose=1 # verbosity mode
)



In [34]:
# Train the model
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=15,
    callbacks=[early_stopping] # add early stopping to the callbacks
)

# Evaluate the model
results = model.evaluate(test_dataset)

# Get the model predictions
predictions = tf.argmax(model.predict(test_dataset)[0], axis=-1)


Epoch 1/15
580/580 [==============================] - 300s 376ms/step - loss: 0.5478 - accuracy: 0.7281 - val_loss: 0.4582 - val_accuracy: 0.7931
Epoch 2/15
580/580 [==============================] - 195s 336ms/step - loss: 0.4331 - accuracy: 0.8062 - val_loss: 0.4529 - val_accuracy: 0.7832
Epoch 3/15
580/580 [==============================] - 192s 332ms/step - loss: 0.3955 - accuracy: 0.8234 - val_loss: 0.4469 - val_accuracy: 0.8014
Epoch 4/15
580/580 [==============================] - 192s 331ms/step - loss: 0.3608 - accuracy: 0.8467 - val_loss: 0.4583 - val_accuracy: 0.8004
Epoch 5/15
580/580 [==============================] - 209s 361ms/step - loss: 0.3264 - accuracy: 0.8657 - val_loss: 0.4866 - val_accuracy: 0.7847
Epoch 6/15
580/580 [==============================] - 192s 330ms/step - loss: 0.2819 - accuracy: 0.8909 - val_loss: 0.4960 - val_accuracy: 0.7968
Epoch 6: early stopping
54/54 [==============================] - 8s 88ms/step


In [35]:
from sklearn.metrics import classification_report


# Decode the label numbers back into original labels
y_test_inverse = encoder.inverse_transform(y_test)
predictions_inverse = encoder.inverse_transform(predictions.numpy())

# Generate the classification report
report = classification_report(y_test_inverse, predictions_inverse,
                               target_names=encoder.classes_.astype(str),digits =4)
print(report)


              precision    recall  f1-score   support

           0     0.8704    0.8774    0.8739       620
           1     0.6766    0.6625    0.6695       240

    accuracy                         0.8174       860
   macro avg     0.7735    0.7700    0.7717       860
weighted avg     0.8163    0.8174    0.8168       860



# test dataset with 150 rows

In [86]:
random_0_rows = test_tweet_a[test_tweet_a['class_a_code'] == 0].sample(n=100, random_state=42)

# Select random 50 rows with class_a_code equal to 1
random_1_rows = test_tweet_a[test_tweet_a['class_a_code'] == 1].sample(n=50)

# Concatenate the two DataFrames
test_tweet_a_150 = pd.concat([random_0_rows, random_1_rows])

In [87]:
test_tweet_a_150= test_tweet_a_150.sample(frac=1).reset_index(drop=True)

In [88]:
X_test_150 = test_tweet_a_150['tweet']
y_test_150 = test_tweet_a_150['class_a_code']

In [89]:
encoder = LabelEncoder()
train_labels = encoder.fit_transform(y_train)

test_labels_150 = encoder.transform(y_test_150)
test_encodings_150 = tokenizer(list(X_test_150), truncation=True, padding=True, max_length=256)


In [90]:
test_dataset_150 = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings_150),
    test_labels_150
)).batch(16)

In [91]:
predictions_150 = tf.argmax(model.predict(test_dataset_150)[0], axis=-1)

10/10 [==============================] - 1s 91ms/step


In [92]:
y_test_inverse_150 = encoder.inverse_transform(y_test_150)
predictions_inverse_150 = encoder.inverse_transform(predictions_150.numpy())

# Generate the classification report
report_150 = classification_report(y_test_inverse_150, predictions_inverse_150,
                               target_names=encoder.classes_.astype(str),digits =4)
print(report_150)

              precision    recall  f1-score   support

           0     0.8750    0.8400    0.8571       100
           1     0.7037    0.7600    0.7308        50

    accuracy                         0.8133       150
   macro avg     0.7894    0.8000    0.7940       150
weighted avg     0.8179    0.8133    0.8150       150



In [94]:
test_tweet_off = test_tweet_a[test_tweet_a['class_a']=='OFF']

In [95]:
test_tweet_off.head()

,tweet,class_a,class_a_code
0,'whoisq wherestheserver dumpnike declasfisa de...,OFF,1
4,'nopasaran unity demo to oppose the farright i...,OFF,1
5,' what the fuck did he do this time',OFF,1
7,' do you get the feeling he is kissing behind...,OFF,1
14,' nigga ware da hits at',OFF,1


In [96]:
test_tweet_off['class_a_code'] =1

<ipython-input-96-3a0f14904e30>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off['class_a_code'] =1


In [98]:
test_tweet_off_00 = test_tweet_off.loc[index_new2]

In [99]:
test_tweet_off_00.shape

(50, 3)

In [100]:
test_tweet_off_00.head()

,tweet,class_a,class_a_code
308,' okay and anybody can go to eredivise and s...,OFF,1
216,'and females b on the same shit ',OFF,1
732,'liberalhypocrisy tacobell when liberals ask w...,OFF,1
376,' fucking years mytwitteranniversary ',OFF,1
382,'christineblaseyford is only being harassed by...,OFF,1


In [102]:
test_tweet_a_150_00 = pd.concat([random_0_rows, test_tweet_off_00])

In [104]:
test_tweet_a_150_00= test_tweet_a_150_00.sample(frac=1).reset_index(drop=True)

In [105]:
X_test_150_00 = test_tweet_a_150_00['tweet']
y_test_150_00 = test_tweet_a_150_00['class_a_code']

In [106]:
encoder = LabelEncoder()
train_labels = encoder.fit_transform(y_train)

test_labels_150_00 = encoder.transform(y_test_150_00)
test_encodings_150_00 = tokenizer(list(X_test_150_00), truncation=True, padding=True, max_length=256)


In [107]:
test_dataset_150_00 = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings_150_00),
    test_labels_150_00
)).batch(16)

In [108]:
predictions_150_00 = tf.argmax(model.predict(test_dataset_150_00)[0], axis=-1)

10/10 [==============================] - 1s 99ms/step


In [110]:
y_test_inverse_150_00 = encoder.inverse_transform(y_test_150_00)
predictions_inverse_150_00 = encoder.inverse_transform(predictions_150_00.numpy())

# Generate the classification report
report_150_00 = classification_report(y_test_inverse_150_00, predictions_inverse_150_00,
                               target_names=encoder.classes_.astype(str),digits =4)
print(report_150_00)

              precision    recall  f1-score   support

           0     0.8155    0.8400    0.8276       100
           1     0.6596    0.6200    0.6392        50

    accuracy                         0.7667       150
   macro avg     0.7376    0.7300    0.7334       150
weighted avg     0.7635    0.7667    0.7648       150



# 125 Test Dataset

In [345]:
random_0_rows = test_tweet_a[test_tweet_a['class_a_code'] == 0].sample(n=100, random_state=42)

# Select random 50 rows with class_a_code equal to 1
random_25_rows = test_tweet_a[test_tweet_a['class_a_code'] == 1].sample(n=25)

# Concatenate the two DataFrames
test_tweet_a_125 = pd.concat([random_0_rows, random_25_rows])

In [346]:
test_tweet_a_125= test_tweet_a_125.sample(frac=1).reset_index(drop=True)

In [347]:
X_test_125 = test_tweet_a_125['tweet']
y_test_125 = test_tweet_a_125['class_a_code']

In [348]:
encoder = LabelEncoder()
train_labels = encoder.fit_transform(y_train)

test_labels_125 = encoder.transform(y_test_125)
test_encodings_125 = tokenizer(list(X_test_125), truncation=True, padding=True, max_length=256)


In [349]:
test_dataset_125 = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings_125),
    test_labels_125
)).batch(16)

In [350]:
predictions_125 = tf.argmax(model.predict(test_dataset_125)[0], axis=-1)

8/8 [==============================] - 1s 91ms/step


In [351]:
y_test_inverse_125 = encoder.inverse_transform(y_test_125)
predictions_inverse_125 = encoder.inverse_transform(predictions_125.numpy())

# Generate the classification report
report_125 = classification_report(y_test_inverse_125, predictions_inverse_125,
                               target_names=encoder.classes_.astype(str),digits =4)
print(report_125)

              precision    recall  f1-score   support

           0     0.9333    0.8400    0.8842       100
           1     0.5429    0.7600    0.6333        25

    accuracy                         0.8240       125
   macro avg     0.7381    0.8000    0.7588       125
weighted avg     0.8552    0.8240    0.8340       125



# Get the test dataset

In [132]:
test_tweet_a_new=pd.read_csv('./testset-levela.tsv', delimiter='\t', encoding='utf-8')
test_label_a_new=pd.read_csv('./labels-levela.csv', encoding='utf-8',
                         index_col=False, names=['id', 'class_a'])

test_tweet_a_new = test_tweet_a_new.merge(test_label_a_new, on='id')

test_tweet_a_new = test_tweet_a_new.drop(columns='id')



In [133]:
test_tweet_a_new

,tweet,class_a
0,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...,OFF
1,"#ConstitutionDay is revered by Conservatives, ...",NOT
2,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...,NOT
3,#Watching #Boomer getting the news that she is...,NOT
4,#NoPasaran: Unity demo to oppose the far-right...,OFF
...,...,...
855,#DespicableDems lie again about rifles. Dem Di...,OFF
856,#MeetTheSpeakers 🙌 @USER will present in our e...,NOT
857,3 people just unfollowed me for talking about ...,OFF
858,#WednesdayWisdom Antifa calls the right fascis...,NOT


In [134]:
off_new = test_tweet_a_new[test_tweet_a_new['class_a']=='OFF']

In [135]:
off_new

,tweet,class_a
0,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...,OFF
4,#NoPasaran: Unity demo to oppose the far-right...,OFF
5,. . . What the fuck did he do this time?,OFF
7,@USER Do you get the feeling he is kissing @US...,OFF
14,@USER Nigga ware da hits at,OFF
...,...,...
842,"#Antifa are mentally unstable cowards, pretend...",OFF
847,@USER @USER And Browning looked like dog shit ...,OFF
853,All two of them taste like ass. URL,OFF
855,#DespicableDems lie again about rifles. Dem Di...,OFF


In [136]:
off_new['tweet'][0]

'#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA Democrats Support Antifa, Muslim Brotherhood, MS13, ISIS, Pedophilia, Child Trafficking, Taxpayer Funded Abortion’s, Election Fraud, Sedition And Treason!!! #LockThemAllUp #WWG1WGA #QAnon \u2066@USER URL'

In [137]:
off_new['tweet'][216]

'And🤷🏽\u200d♂️ females b on the same shit URL'

In [138]:
index_new = [27,370,334,702,857,243,460,827,326,30,221,617,173,85,371,567,428,740,439,408,420,638,663,377,775,502,468,177,180,422,262,335,774,555,657,201,222,511,549,509,587,771,437,800,108,207,609,156,0,857]

In [139]:
test_tweet_off_new =off_new.loc[index_new]

# ChatGPT 3.5 with rephrasing less aggressive

In [140]:
test_tweet_off_new['tweet'][27] ="Let's consider a scenario where DeLauro is put in a police lineup. Some people might point her out due to certain negative perceptions. This situation raises questions about the voters' choices or the possibility of voter fraud. It's essential to remember that everyone's right to vote should be respected, regardless of their gender or appearance. \u2066@USER @USER"

In [141]:
test_tweet_off_new['tweet'][370] ="Recent events involving #Beeto in #Texas serve as a demonstration of how not to run a successful election campaign. #MAGA \u2066@USER #rkba @USER #DVC UR"

In [142]:
test_tweet_off_new['tweet'][334] ="#Conservative Individuals can learn valuable lessons from the gaming community. URL #Conservatives #TriggerWarningRadio"

In [143]:
test_tweet_off_new['tweet'][702] = "I'd like to share my opinion, even though #NoOneAskMe. Regarding the ongoing beef between #CardiB and @USER, it seems like #Offset is involved to some extent. Cardi might be trusting him, considering their relationship. However, it's important for #offset to handle the situation calmly and thoughtfully."

In [144]:
test_tweet_off_new['tweet'][857]="Three people recently unfollowed me after discussing Merlin. Apologies, everyone! I'm still obsessed with Covinsky, but I'm eagerly waiting for a possible sequel announcement for PS I Still Love You. I've run out of witty and funny tweets about To All the Boys I've Loved Before; it's been quite exhausting."

In [145]:
test_tweet_off_new['tweet'][243] = "#Trump's actions have raised concerns about possible obstruction of justice, as some see them as significant evidence. Critics view him as a controversial figure in this regard."

In [146]:
test_tweet_off_new['tweet'][460] = "#Feinstein, it's important to be honest and truthful in your statements. Let's strive to maintain integrity and avoid spreading misinformation. URL"

In [147]:
test_tweet_off_new['tweet'][827] = "#MAGA Can you please share one positive action that #Trump has taken during his time as #president? I'm open to hearing different perspectives. Let's discuss his accomplishments and record objectively."

In [148]:
test_tweet_off_new['tweet'][326] = "#FamiliarWife JH-Oh my goodness, she's quite unpredictable. WJ-Hold on!...yes, I find assistant manager Cha interesting. BH-You two seem compatible. VO-Any issues between you? Just sit here. JH-I advise against it! WJ-Let's live life on our terms. Let's give it a try. News-There's a collision on the highway causing traffic. JH-Woo-jin~ URL"

In [149]:
test_tweet_off_new['tweet'][30] = "#ChristineBlaseyFord is the individual who accused #Kavanaugh. It seems like this accusation is a pattern that #Liberals tend to follow each time. Let's focus on discussing the #ConfirmJudgeKavanaugh process. URL"


In [150]:
test_tweet_off_new['tweet'][221]="#DeclassifyDocuments It's intriguing to observe different perspectives, like Maria's, during discussions. #Outnumbered"

In [152]:
test_tweet_off_new['tweet'][617] = "#LiberalismIsAMentalDisorder It's essential to recognize differing viewpoints. #Hypocrites #DemocratsHateAmerica #DemocratsAreDangerous It's important to engage in constructive dialogue and find common ground. URL"

In [153]:
test_tweet_off_new['tweet'][173] ="#Elitist #Actors #NBC #Liberals at the #Emmy Awards showcased a self-promotion event with some #MeToo references. It's worth noting that this was a planned and staged occasion, which might align with their #Democratic and #Socialist beliefs. Let's remember the importance of respectful dialogue and understanding different perspectives. #WALKAWAY URL"

In [154]:
test_tweet_off_new['tweet'][85] = "#Liberals can be seen differently by some, but let's remember the principle of INNOCENT UNTIL PROVEN GUILTY. It's important to examine both parties if there are concerns about individuals' conduct. Let's strive to have respectful discussions and avoid harsh language. #KavanaughIsInnocent #WalkAwayFromDemocrats. Let's focus on honesty and avoid extreme accusations. URL"

In [155]:
test_tweet_off_new['tweet'][371] ="And sometimes, people can give off the impression that they have money, but in reality, they may not"

In [156]:
test_tweet_off_new['tweet'][567] ="Ford's attorney has mentioned that she's facing death threats, leading her to relocate her family for safety. However, it's essential to seek evidence before making assumptions about who might be behind these threats. Let's approach this situation with a fair and unbiased perspective"

In [157]:
test_tweet_off_new['tweet'][428]="Anyone who finds M*neta's behavior humorous may have a different sense of humor."

In [158]:
test_tweet_off_new['tweet'][740] = "@USER - Opinions about this POTUS vary widely."

In [159]:
test_tweet_off_new['tweet'][439] = "#CnnIsFakeNews @USER @USER, some people have concerns about the network's ratings and credibility. #cnnisajoke #FakeNews #MAGA URL"

In [160]:
test_tweet_off_new['tweet'][408] = "#Hillary and at least 16 other individuals have strong opinions as #AngryDemocrats. Let's discuss #WorldWarQ, #WWG1WGA, #MAGA, #RedTsunami, #AmericaFirst, #WalkAway, and #Truth. URL"

In [161]:
test_tweet_off_new['tweet'][420] ="@USER @USER As I've mentioned before, some people believe that media and financial influences, like Soros and Waters, are contributing to increased tensions in society."

In [162]:
test_tweet_off_new['tweet'][638] ="#Liberals are often known for their strategic tactics to achieve their goals. 👇👇 URL"

In [163]:
test_tweet_off_new['tweet'][663]="#HonourSacrificeBoycottMatch I have concerns about the impact on human rights. I hope all parties involved find some satisfaction. Let's engage in cricket with them and express gratitude for their efforts to support our soldiers. We need to address the situation thoughtfully; even smaller nations like Pakistan should not be underestimated."

In [164]:
test_tweet_off_new['tweet'][377] = "#Dems have shown a concerning pattern of believing accusations against conservatives while dismissing those against liberals. There is a need for fair and impartial treatment in such cases. URL"

In [165]:
test_tweet_off_new['tweet'][775]="#FemiNazis, who are often associated with Liberals, seem to emphasize #MeToo only when accusing non-Liberal men. The situation may seem perplexing. #Bizarro #KAG2018 ❤️🇺🇸 URL"

In [166]:
test_tweet_off_new['tweet'][502] ="@USER One possible explanation is that he may be tone-deaf to the situation. It's challenging to remain understanding when he and others continue to behave this way."

In [167]:
test_tweet_off_new['tweet'][468] ="#Nigeria #Naija #9ja 'Atiku criticizes Buhari, calling him one of the least competent leaders in Nigeria's history' - Read more on this via URL provided by championnewsng."

In [168]:
test_tweet_off_new['tweet'][177] ="#BlueWave #DumpTrump ...show love and seek to cooperate or find common ground with conservatives/Trump supporters while advocating for the rights of marginalized groups and supporting women's rights to make choices about their own bodies..."

In [169]:
test_tweet_off_new['tweet'][180] ="#NightMayor's DT cycle track can be repurposed for various community activities, like setting up tennis courts for Antifa inside Crescent Heights, until we address concerns related to Nenshi's vape shop."

In [170]:
test_tweet_off_new['tweet'][422] ="#Texas Ted Cruz's campaign has been sending solicitations that some recipients mistake for official county summons. This has caused confusion, particularly among elderly residents. Some people find this approach questionable. URL"

In [171]:
test_tweet_off_new['tweet'][262] = "$uicideboy$ ON HALLOWEEN! Get ready for a seriously spooky time!"

In [172]:
test_tweet_off_new['tweet'][335] ="#Barbara Boxer: When discussing serious issues like rape, it's crucial for everyone, including liberals, to provide solid evidence. Personal accounts alone might not be sufficient. On a related note, without evidence, making accusations could be misleading. Let's focus on respectful discussions. URL"

In [173]:
test_tweet_off_new['tweet'][774] ="@USER #Rosie's actions or statements sometimes make me feel uncomfortable. URL"

In [174]:
test_tweet_off_new['tweet'][555] ="Anyway, this game isn't really my cup of tea :)"

In [175]:
test_tweet_off_new['tweet'][657] ="#Bakersfield is an example of why some people advocate for gun control. My thoughts are with the victims, except for the gunman himself. I hope justice is served"

In [176]:
test_tweet_off_new['tweet'][201] ="@USER These seem to be coordinated attacks from individuals with extreme views. It's unfortunate that such incidents may escalate further before the election. Some people criticize the President for not being a traditional politician, even though they fear potential consequences for the economy and their 401 plans. Let's hope for more reasoned and respectful discussions."

In [177]:
test_tweet_off_new['tweet'][222] ="#PoliticsInFocus Hey Donald! Let's remember we all have different perspectives, no need to talk down to me! URL'"

In [178]:
test_tweet_off_new['tweet'][511] ="#Kavanaugh has been influenced by Anthony Kennedy, known for delivering opinions that upset liberals. Some view him as an attempt by Trump to convince conservatives that the #SCOTUS is leaning right. If @USER were strategic, they would work to support BK's confirmation. However, it seems unlikely. #TuesdayThoughts URL"

In [179]:
test_tweet_off_new['tweet'][549] ="#NEWS Jeff Sessions: In his perspective, following the guidance of the ACLU, Antifa, and Black Lives Matter may lead to more negative outcomes. URL #CNBC"

In [180]:
test_tweet_off_new['tweet'][509] = "...how about death metal? I'm eager to discover more death metal music... Can you please suggest some good places to find it? URL"

In [181]:
test_tweet_off_new['tweet'][587] ="@USER #ThoseThatAreTheLoudest might be concerned, but many believe that justice will prevail. Let's remember to focus on the facts. #Bono and the #ClintonFoundation have raised some questions, and people have expressed skepticism about their actions. #PedoWood #Haiti URL"

In [182]:
test_tweet_off_new['tweet'][771] = "#Conservatives might not be as informed as some may hope, that's the main idea behind my tweet."

In [183]:
test_tweet_off_new['tweet'][437] = "#MSNBC #Hardball @USER Bingo, you've hit the mark. It seems evident that he holds a grudge against President Obama, likely stemming from the White House correspondents dinner. This could be a significant factor in his negative feelings towards the former President."

In [184]:
test_tweet_off_new['tweet'][800] ="#Kavanaugh If #BlaseyFord's credibility is questioned, there's a possibility of another accuser coming forward. Some people criticize liberals for their consistent approach in such situations. #LiberalismIsAMentalDisorder"

In [185]:
test_tweet_off_new['tweet'][108] = "Amazon is now offering to ship live Christmas trees to your doorstep. But the big question is, will they stay and help with putting on the lights? URL"

In [186]:
test_tweet_off_new['tweet'][207] ="#JoinTheFight Join the discussion about concerns regarding certain political ideologies. Some people feel that Democrats and Liberals are promoting ideas that they believe could be harmful to America. Let's engage in constructive conversations. #JoinTheFightAgainstCommunism URL"

In [187]:
test_tweet_off_new['tweet'][609] ="#DeSantis=DeRacist, while some may view him as a questionable choice, he chose to distance himself from the President's comments about deaths in Puerto Rico from Hurricane Maria. @USER @USER #whataputz"

In [188]:
test_tweet_off_new['tweet'][156] ="#TheArchers I want to excel in Karate and improve my skills for self-defense and personal growth."

In [193]:
test_tweet_off_new.shape

(50, 2)

In [194]:
test_tweet_not = test_tweet_a_new[test_tweet_a_new['class_a']=='NOT']

In [195]:
random_100_rows = test_tweet_not.sample(n=100)

# Build the new DataFrame using the randomly selected rows
test_tweet_not_new = pd.DataFrame(random_100_rows)

In [196]:
test_tweet_not_new

,tweet,class_a
516,#Trump #MAGA| @USER @USER on #ElizabethSmart c...,NOT
498,#PiperThePup sends her love to everyone who mi...,NOT
92,"#FordTheatre Right on Cue, a GoFundMe acct ha...",NOT
619,...veer left by the old church...,NOT
257,An open letter to Mac Miller URL,NOT
...,...,...
586,10 minutes in to Instinct and I love it #AlanC...,NOT
166,#Eagles are setting up a workout with Jeremy M...,NOT
174,@USER Kellerman knows only 1 thing more then t...,NOT
804,Aparently my computer doesn't want to operate ...,NOT


In [197]:
merged_df = pd.concat([test_tweet_not_new, test_tweet_off_new], ignore_index=True)

# Shuffle the index
test_shuffled_df = merged_df.sample(frac=1).reset_index(drop=True)

In [198]:
test_shuffled_df

,tweet,class_a
0,"3,000 is definitely a tad high Puerto Rico. Ho...",NOT
1,#WCW #WCE @USER It’s your special day of the ...,NOT
2,"#LisaxMichaelKors she is sooo talented, love h...",NOT
3,@USER These seem to be coordinated attacks fro...,OFF
4,#Trump is going to visit victims of #Hurricane...,NOT
...,...,...
145,#Charlotte could see anywhere from 6 to 10” of...,NOT
146,#MAGA Can you please share one positive action...,OFF
147,#Conservatives might not be as informed as som...,OFF
148,"#CnnIsFakeNews @USER @USER, some people have c...",OFF


In [199]:
clean_tweets(test_shuffled_df)

<ipython-input-10-58d0d99cd0a4>:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, 'tweet'] = df.tweet.str.replace('\d+','') #Remove numbers
<ipython-input-10-58d0d99cd0a4>:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.loc[:, 'tweet'] = df.tweet.str.replace(punctuation, '')


In [200]:

test_shuffled_df.update(test_shuffled_df[['tweet']].applymap('\'{}\''.format))

test_shuffled_df.head()

,tweet,class_a
0,' is definitely a tad high puerto rico how abo...,NOT
1,'wcw wce its your special day of the week ag...,NOT
2,'lisaxmichaelkors she is sooo talented love hr...,NOT
3,' these seem to be coordinated attacks from in...,OFF
4,'trump is going to visit victims of hurricanef...,NOT


In [201]:

test_shuffled_df["class_a_code"] = encoder.fit_transform(test_shuffled_df["class_a"])

In [202]:
X_test_shuffled = test_shuffled_df['tweet']
y_test_shuffled = test_shuffled_df['class_a_code']

In [203]:
X_test_shuffled

0      ' is definitely a tad high puerto rico how abo...
1      'wcw wce   its your special day of the week ag...
2      'lisaxmichaelkors she is sooo talented love hr...
3      ' these seem to be coordinated attacks from in...
4      'trump is going to visit victims of hurricanef...
                             ...                        
145    'charlotte could see anywhere from  to  of rai...
146    'maga can you please share one positive action...
147    'conservatives might not be as informed as som...
148    'cnnisfakenews   some people have concerns abo...
149    'liberalismisamentaldisorder its essential to ...
Name: tweet, Length: 150, dtype: object

In [204]:
y_test_shuffled

0      0
1      0
2      0
3      1
4      0
      ..
145    0
146    1
147    1
148    1
149    1
Name: class_a_code, Length: 150, dtype: int64

In [205]:
encoder = LabelEncoder()
train_labels = encoder.fit_transform(y_train)

test_labels_shuffled = encoder.transform(y_test_shuffled)

In [206]:
test_encodings_shuffled = tokenizer(list(X_test_shuffled), truncation=True, padding=True, max_length=256)

In [207]:
test_dataset_shuffled = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings_shuffled),
    test_labels_shuffled
)).batch(16)

In [208]:

# Get the model predictions
predictions = tf.argmax(model.predict(test_dataset_shuffled)[0], axis=-1)


10/10 [==============================] - 1s 90ms/step


In [209]:
# Decode the label numbers back into original labels
y_test_inverse_shuffled = encoder.inverse_transform(y_test_shuffled)
predictions_inverse_shuffled = encoder.inverse_transform(predictions.numpy())

# Generate the classification report
report_gpt3 = classification_report(y_test_inverse_shuffled, predictions_inverse_shuffled,
                               target_names=encoder.classes_.astype(str),digits =4)
print(report_gpt3)

              precision    recall  f1-score   support

           0     0.6471    0.8800    0.7458       100
           1     0.1429    0.0400    0.0625        50

    accuracy                         0.6000       150
   macro avg     0.3950    0.4600    0.4041       150
weighted avg     0.4790    0.6000    0.5180       150



# ChatGPT 3 with rephrasing (150)


In [210]:
index_new2 = [308, 216, 732, 376, 382, 339, 47, 99, 797, 282, 228, 362, 701, 480, 115, 136, 77, 51, 681, 749, 389, 260, 258, 100, 233, 554, 810, 483, 614, 778, 107, 613, 175, 307, 634, 313, 182, 154, 855, 352, 853, 323, 40, 38, 531, 135, 281, 737, 823, 413]

In [211]:
test_tweet_off_new = off_new.loc[index_new2]

In [212]:
test_tweet_off_new['tweet'][413]

'#Antifa is recruiting for a designated #terrorist organization URL'

In [213]:
test_tweet_off_new['tweet'][308] = "Alright, let's consider the situation. It's true that some players have scored 15+ goals in Eredivisie, but some might argue that it's not the most challenging league. On the other hand, Bundesliga teams have proven to be competitive in European competitions. As for Mexican talent, it's been some time since someone like Pulisic emerged. Nevertheless, there's always hope for the future, and who knows, Mexico might produce exceptional talents in due time."

In [214]:
test_tweet_off_new['tweet'][216] ="It seems like both males and females are dealing with similar situations, and that's something to consider. The URL might be a shared experience for both genders."

In [215]:
test_tweet_off_new['tweet'][732] = "Addressing the topic of #LiberalHypocrisy and #TacoBell, some individuals question why others are against illegal immigration. They mention an incident involving a Taco Bell employee responding with 'No Habla Ingles!' when asked."

In [216]:
test_tweet_off_new['tweet'][376]  ="It's been an incredible seven years! #MyTwitterAnniversary URL"

In [217]:
test_tweet_off_new['tweet'][382] ="The situation surrounding #ChristineBlaseyFord has drawn attention, and some suggest that she is facing harassment from certain groups. They point to the involvement of Antifa, seen as the radical arm of the Democratic Party, allegedly backed by George Soros. The hashtag #VoteRedToSaveAmerica is being used in this context."

In [218]:
test_tweet_off_new['tweet'][339] ="On #ConstitutionDay, there's an interesting contrast between some conservative members of the alt-right who criticize efforts for #GunControlNow, accusing it of damaging the Constitution, while expressing concerns about non-White foreigners seeking to live in this country. The situation sparks debate on constitutional interpretations and immigration policies."

In [219]:
test_tweet_off_new['tweet'][47] ="#Conservatives, it's disheartening to witness such disrespectful interactions like the one with @USER. URL"

In [220]:
test_tweet_off_new['tweet'][99]  ="Some people find #SerenaWilliams to be overly confident, making it challenging to enjoy both watching her play and listening to her speak."

In [221]:
test_tweet_off_new['tweet'][797] ="If you want to encourage people to leave your home, perhaps you could try using humor or lighthearted jokes, like pretending to be a vampire, without causing harm or discomfort. It's essential to find a friendly and non-threatening way to address the situation."

In [222]:
test_tweet_off_new['tweet'][282] = "The ongoing #CrowderConfronts series continues to demonstrate that when conservatives respond with truth and facts, the so-called SJW bullies appear less intimidating and substantial. It encourages others not to be afraid and emphasizes the importance of confronting issues with honesty and evidence. URL"

In [223]:
test_tweet_off_new['tweet'][228] ="'#CarbonTax proves effective and promises to significantly increase money in the pockets of Canadians. .@USER .@USER .@USER .@USER Back in 2015, part of Trudeau's successful campaign was centered around a national carbon tax, which played a role in the Conservative party's defeat. They face a similar challenge in 2019, and some are determined to ensure their loss once again. URL'"

In [224]:
test_tweet_off_new['tweet'][362] ="'#Repost theleadersmindset • • • 📍Hello Leaders!🔥 ✔️Type 'READY' if you are prepared to embrace these steps. - 'Set goals, keep them to yourself, crush them with determination, celebrate your successes… URL'"

In [225]:
test_tweet_off_new['tweet'][701] ="'@USER nigga are you stupid your trash dont play with him play with your bitch 😂'"

In [226]:
test_tweet_off_new['tweet'][480] ="#FailedDemFeinstein's continued presence in politics is questionable, and some believe she should have stepped down years ago. Her recent actions, like blaming the GOP in the Kavanaugh accuser situation while admitting uncertainty about the truthfulness, add to her perceived disgrace. URL"

In [227]:
test_tweet_off_new['tweet'][115] ="#Conservatives Govt ' @USER caused immense hardship': whistleblowers report being threatened by a regulator; Weren't they supposed to safeguard whistleblowers? URL"

In [228]:
test_tweet_off_new['tweet'][136] ="@USER #WalkAwayFromAllDemocrats Michelle had her 8 years, and many believe we don't need more of her advice. We now have a new Sheriff in Town who's #MAGA. It's crucial for everyone, regardless of color or gender (AKA all Americans), to get out and VOTE! URL"

In [229]:
test_tweet_off_new['tweet'][77] ="#ANTIFA is seen as allied with The Democratic Socialists of America (DSA), whom #JamesOKeefe has revealed as having infiltrated various government positions with the aim of undermining #America. #Trump #MAGA🇺🇸 #DeepState URL"

In [230]:
test_tweet_off_new['tweet'][51] ="'#Kids Beating #Paraplegic #Trump #Boy!  #Left Teach Their #Children Hate Violence!  #schools #teachers #Democrats #Liberals #WalkAway #Moms #Mother #NEA #teachthemyoung #Christian #Catholic #Jews #Evangelicals #family #BlackWomen #BlackTwitter #Blacks #News #Media #Disabled #MAGA URL'"

In [231]:
test_tweet_off_new['tweet'][681] ="Can you believe it? A hundred rounds! Goon, you must be kidding me! 🤣"

In [232]:
test_tweet_off_new['tweet'][749] ="#Antifa plans to counter the fascist NVU in #Amsterdam on September 22. URL"

In [233]:
test_tweet_off_new['tweet'][389] ="50 Cent addresses Joe Budden's actions on Instagram, calling them out as 'bullshit.' URL URL"

In [234]:
test_tweet_off_new['tweet'][260] ="#Liberals are resorting to extreme measures, urging #PhillipRuddock to communicate with #Turnbull and persuade him to assist with #WentworthVotes on 18 Sept 2018. @USER #Auspol #LNP #NSWpol @USER @USER @USER #LNPMemes URL"

In [235]:
test_tweet_off_new['tweet'][258] ="'#CincinnatiEnquirer finally addresses #GunControl following a tragic #MassShooting right on their doorstep. It took this event for their editors to wake up, but better late than never. URL'"

In [236]:
test_tweet_off_new['tweet'][100] ="Regarding #ColinKaepernick, some people believe he has been unproductive and merely voicing complaints. They think it's time for both black individuals and passionate white liberals, including @USER, to become more aware or stop making noise about the cause. They see Colin as not fitting the role of a Civil Rights hero or martyr by any means."

In [237]:
test_tweet_off_new['tweet'][233] ="During #TheView, Stormy Daniels found herself in a difficult situation with a dollar bill in her face, highlighting the challenges faced by adult film stars. The post also seems to allude to her involvement with the Democratic Party."

In [238]:
test_tweet_off_new['tweet'][554] = "#DavidMiliband relocated to #NewYork to align with #US #Liberals #Democrats, seeking a departure from the hostility towards Jews by #Corbynists."

In [239]:
test_tweet_off_new['tweet'][810] ="'#BCOG @USER portrays a cool, confident, and assertive character who is highly skilled and successful! @USER @USER @USER URL'"

In [240]:
test_tweet_off_new['tweet'][483] ="'@USER @USER @USER Your meme suggests that both Fascism and Antifa were opposed to liberals.'"

In [241]:
test_tweet_off_new['tweet'][614] ="@USER @USER I plan to use that argument in future discussions on gun control, free speech, or taxes. It's essential to promote responsibility, and that's what I advocate for. There's nothing right about justifying murder by denying the life of another or claiming it as a right."

In [242]:
test_tweet_off_new['tweet'][778] ="It appears that a #BREXIT deal has been reached and will be presented at a special summit in November. The question raised is whether @USER has compromised the #UK's position with the EU. People are concerned, hoping that there hasn't been a sellout, as it could have significant consequences for @USER. URL"

In [243]:
test_tweet_off_new['tweet'][107] ="Jeff Sessions suggests that listening to groups like the ACLU, Antifa, and Black Lives Matter could result in an increase in Chicago shootings. URL"

In [244]:
test_tweet_off_new['tweet'][613] ="Lost my acca on the first fight, 0-1. So frustrating."

In [245]:
test_tweet_off_new['tweet'][307] ="#1 son, hitting it out of the park! URL"

In [246]:
test_tweet_off_new['tweet'][313]  = "@USER @USER @USER Many believe she is the most disingenuous person in the Senate. If she were my Senator, I would feel ashamed due to her despicable behavior in the SCOTUS hearings."

In [247]:
test_tweet_off_new['tweet'][182] = "#ArunJaitleyStepDown is trending, with many expressing strong criticism towards him. They consider him the most shameless Finance Minister in the history of India, and his audacity to lie in public is viewed as a disgrace to his position."

In [248]:
test_tweet_off_new['tweet'][154] ="#GOP still appearing hesitant, and some view them as lacking courage. It seems they haven't realized that Liberals will oppose them no matter what they say. Some believe Republicans are either struggling to make strategic decisions or have shifted too far left. #ConfirmKavanaughNow"

In [249]:
test_tweet_off_new['tweet'][855] = "'#DespicableDems are accused of spreading misinformation about rifles. Some claim that Democrats manipulated the law to advocate for gun control during the Kavanaugh confirmation. URL via @USER'"

In [250]:
test_tweet_off_new['tweet'][352] = "'#OrrinHatch's involvement in another woman's charges against a Supreme Court nominee has sparked strong criticism. Some view him as sexist, clueless, and outdated, drawing comparisons to his past behavior during the Anita Hill case. They believe it's time for him to step aside. Good riddance to this Neanderthal!'"

In [251]:
test_tweet_off_new['tweet'][853] ="Both of them have an unpleasant taste. URL"

In [252]:
test_tweet_off_new['tweet'][323] = "#AVALANCHE, I apologize! I acknowledge your presence, and I care about you! I regret my previous behavior and acknowledge that we all make mistakes. #maga"

In [253]:
test_tweet_off_new['tweet'][38] = "#ArianaAsesina? Seriously?! Please, let's not jump to conclusions and spread hate. She's already going through a tough time, and pointing fingers won't help. It was an overdose, and blaming others won't change the tragic outcome. Let's stop seeking someone to blame, and instead, show empathy and support."



In [254]:
test_tweet_off_new['tweet'][531] = "@USER, it's widely known that a genuine leader doesn't stumble or fall like a sack of potatoes into a van..."

In [255]:
test_tweet_off_new['tweet'][135] = "'#BoycottNFL, I have decided to no longer support you. You have lost my loyalty. @USER @USER URL'"

In [256]:
test_tweet_off_new['tweet'][281]  = "'#Conservatives Govt has accumulated debt despite implementing austerity cuts, and during this time, the wealth of the rich has doubled. This highlights the issue of #inequality. URL via @USER'"

In [257]:
test_tweet_off_new['tweet'][413] = "'#Antifa is actively seeking new members for what some perceive as a designated #terrorist organization. URL'"

In [258]:

merged_df_02 = pd.concat([test_tweet_not_new, test_tweet_off_new], ignore_index=True)

# Shuffle the index
test_02_df = merged_df_02.sample(frac=1,random_state=42).reset_index(drop=True)

In [259]:
test_02_df

,tweet,class_a
0,#Florencehurricane 72 HOURS OF SUSTAINED LETHA...,NOT
1,#Angel_Dad: If Lawmakers Really Cared About Ci...,NOT
2,"Can you believe it? A hundred rounds! Goon, yo...",OFF
3,#TheNunMovie was just as scary as I thought it...,NOT
4,@USER #Google as a fellow Goggler I must say e...,NOT
...,...,...
145,"#KamalaHarris may run for President in 2020, b...",NOT
146,"#Conservatives, it's disheartening to witness ...",OFF
147,#FridayNightFun!! With Halloween events star...,NOT
148,@USER #GoodbyeNFL 👋 It was great before your p...,NOT


In [260]:
clean_tweets(test_02_df)

<ipython-input-10-58d0d99cd0a4>:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, 'tweet'] = df.tweet.str.replace('\d+','') #Remove numbers
<ipython-input-10-58d0d99cd0a4>:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.loc[:, 'tweet'] = df.tweet.str.replace(punctuation, '')


In [261]:
test_02_df.update(test_02_df[['tweet']].applymap('\'{}\''.format))

test_02_df.head()

,tweet,class_a
0,'florencehurricane hours of sustained lethal ...,NOT
1,'angeldad if lawmakers really cared about citi...,NOT
2,'can you believe it a hundred rounds goon you ...,OFF
3,'thenunmovie was just as scary as i thought it...,NOT
4,' google as a fellow goggler i must say electi...,NOT


In [262]:

test_02_df["class_a_code"] = encoder.fit_transform(test_02_df["class_a"])

In [263]:
X_test_02 = test_02_df['tweet']
y_test_02 = test_02_df['class_a_code']

In [264]:
encoder = LabelEncoder()
train_labels = encoder.fit_transform(y_train)

test_labels_02 = encoder.transform(y_test_02)

In [265]:
test_encodings_02 = tokenizer(list(X_test_02), truncation=True, padding=True, max_length=256)

In [266]:
test_dataset_02 = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings_02),
    test_labels_02
)).batch(16)

In [267]:
# Get the model predictions
predictions_02 = tf.argmax(model.predict(test_dataset_02)[0], axis=-1)

10/10 [==============================] - 1s 101ms/step


In [268]:
# Decode the label numbers back into original labels
y_test_inverse_02 = encoder.inverse_transform(y_test_02)
predictions_inverse_02 = encoder.inverse_transform(predictions_02.numpy())

# Generate the classification report
report_02 = classification_report(y_test_inverse_02, predictions_inverse_02,
                               target_names=encoder.classes_.astype(str),digits =4)
print(report_02)

              precision    recall  f1-score   support

           0     0.7273    0.8800    0.7964       100
           1     0.5862    0.3400    0.4304        50

    accuracy                         0.7000       150
   macro avg     0.6567    0.6100    0.6134       150
weighted avg     0.6803    0.7000    0.6744       150



# gpt 3.5 rephrase with 125

In [417]:
index_new5 = random.sample(index_new2, 25)

In [418]:
test_tweet_off_new5 = test_tweet_off_new.loc[index_new5]

In [419]:
test_tweet_off_new5.shape

(25, 2)

In [420]:

merged_df_05 = pd.concat([test_tweet_not_new, test_tweet_off_new5], ignore_index=True)

# Shuffle the index
test_05_df = merged_df_05.sample(frac=1).reset_index(drop=True)

In [421]:
clean_tweets(test_05_df)

<ipython-input-10-58d0d99cd0a4>:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, 'tweet'] = df.tweet.str.replace('\d+','') #Remove numbers
<ipython-input-10-58d0d99cd0a4>:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.loc[:, 'tweet'] = df.tweet.str.replace(punctuation, '')


In [422]:
test_05_df.update(test_05_df[['tweet']].applymap('\'{}\''.format))

In [423]:

test_05_df["class_a_code"] = encoder.fit_transform(test_05_df["class_a"])

In [424]:
X_test_05 = test_05_df['tweet']
y_test_05 = test_05_df['class_a_code']

In [425]:
encoder = LabelEncoder()
train_labels = encoder.fit_transform(y_train)

test_labels_05 = encoder.transform(y_test_05)

In [426]:
test_encodings_05 = tokenizer(list(X_test_05), truncation=True, padding=True, max_length=256)

In [427]:
test_dataset_05 = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings_05),
    test_labels_05
)).batch(16)

In [428]:
# Get the model predictions
predictions_05 = tf.argmax(model.predict(test_dataset_05)[0], axis=-1)

8/8 [==============================] - 1s 105ms/step


In [429]:
# Decode the label numbers back into original labels
y_test_inverse_05 = encoder.inverse_transform(y_test_05)
predictions_inverse_05 = encoder.inverse_transform(predictions_05.numpy())

# Generate the classification report
report_05 = classification_report(y_test_inverse_05, predictions_inverse_05,
                               target_names=encoder.classes_.astype(str),digits =4)
print(report_05)

              precision    recall  f1-score   support

           0     0.8544    0.8800    0.8670       100
           1     0.4545    0.4000    0.4255        25

    accuracy                         0.7840       125
   macro avg     0.6545    0.6400    0.6463       125
weighted avg     0.7744    0.7840    0.7787       125



# ChatGPT 4 with rephrasing 125

In [269]:
index_new3 = random.sample(index_new2, 25)

In [271]:
test_tweet_off_new_4 = off_new.loc[index_new3]

In [272]:
test_tweet_off_new_4['tweet'][853]


'All two of them taste like ass. URL'

In [273]:
test_tweet_off_new_4['tweet'][135] ="#AvoidNFL, I've chosen to withdraw my support. My loyalty has been lost. @USER @USER [URL]"

In [274]:
test_tweet_off_new_4['tweet'][216] = "Both men and women appear to face similar circumstances, which is worth noting. The URL could reflect an experience common to both genders."

In [275]:
test_tweet_off_new_4['tweet'][308] ="Okay, let's evaluate the scenario. While certain players have netted more than 15 goals in the Eredivisie, there are those who believe it's not the toughest league. Conversely, Bundesliga squads have demonstrated their mettle in European tournaments. Regarding Mexican players, it's been a while since we've seen a standout like Pulisic. However, the future remains promising, and Mexico could very well nurture outstanding players in the coming years."

In [276]:
test_tweet_off_new_4['tweet'][389] ="50 Cent calls out Joe Budden's behavior on Instagram, labeling it as 'nonsense.' [URL] [URL]"

In [277]:
test_tweet_off_new_4['tweet'][823] = "#DemocraticPartyConcerns #Extremists #AntifaIssues #RadicalFeminism Consider distancing yourself. [URL]"

In [278]:
test_tweet_off_new_4['tweet'][115] = "#ConservativeGovernment - Whistleblowers claim they faced threats under @USER's regime, despite regulators supposed to protect them. [URL]"

In [279]:
test_tweet_off_new_4['tweet'][323] ="#AVALANCHE, my sincere apologies! I recognize and value you. Regretfully, I made errors in the past, but we all err at times. #maga"

In [280]:
test_tweet_off_new_4['tweet'][77] = "#ANTIFA is perceived to be associated with The Democratic Socialists of America (DSA). #JamesOKeefe has exposed their alleged presence in multiple government roles, purportedly aiming to challenge #America's foundations. #Trump #MAGA🇺🇸 #UnderlyingForces [URL]"

In [281]:
test_tweet_off_new_4['tweet'][260] ="#Liberals are taking significant steps, asking #PhillipRuddock to liaise with #Turnbull in hopes of influencing the #WentworthVotes on 18 Sept 2018. @USER #Auspol #LNP #NSWpol @USER @USER @USER #LNPTrends [URL]"

In [282]:
test_tweet_off_new_4['tweet'][281] ="#Conservatives Government, despite austerity measures, has seen an increase in debt. Meanwhile, the affluent's wealth has grown twofold, spotlighting the #inequality problem. [URL] courtesy of @USER."

In [283]:
test_tweet_off_new_4['tweet'][352]  ="#OrrinHatch's role in handling another woman's allegations against a Supreme Court nominee has garnered significant backlash. Some consider his approach reminiscent of his stance during the Anita Hill case, labeling him as out of touch. Many feel it's time for a change in leadership."

In [284]:
test_tweet_off_new_4['tweet'][47]  ="#Conservatives, it's unfortunate to see such discourteous exchanges, especially ones involving @USER. [URL]"

In [285]:
test_tweet_off_new_4['tweet'][233] ="On #TheView, Stormy Daniels encountered a challenging moment when confronted with a dollar bill, underscoring the difficulties adult film actors often face. The mention also hints at her ties to the Democratic Party."

In [286]:
test_tweet_off_new_4['tweet'][38] ="Using the hashtag #ArianaAsesina? Let's avoid rushing to judgments and promoting negativity. She's facing difficult moments, and assigning blame doesn't alter the sad reality. An overdose occurred, and finding fault won't help. Let's prioritize understanding and compassion over blame."

In [287]:
test_tweet_off_new_4['tweet'][614] ="@USER @USER I intend to incorporate that perspective in upcoming debates about gun control, free speech, and taxes. Advocating for responsibility is crucial. It's incorrect to excuse taking a life by negating its value or asserting it as an entitlement."

In [288]:
test_tweet_off_new_4['tweet'][100] = "Concerning #ColinKaepernick, there's a viewpoint that he has been more vocal than effective. Some argue that both the black community and supportive white liberals, including @USER, should be more informed or reduce the vocal intensity about the issue. They don't see Colin as embodying the image of a Civil Rights champion."

In [289]:
test_tweet_off_new_4['tweet'][376] = "What an amazing seven-year journey it's been! #MyTwitterAnniversary [URL]."

In [290]:
test_tweet_off_new_4['tweet'][681] = "Can you even imagine? A hundred rounds? You've got to be joking, Goon! 🤣"

In [291]:
test_tweet_off_new_4['tweet'][797]  = "To get folks to head out from your place, consider a humorous touch, like playfully acting like a vampire. Just ensure it's all in good fun and doesn't make anyone uneasy. Always approach situations with kindness and without being intimidating."

In [292]:
test_tweet_off_new_4['tweet'][749] = "#Antifa intends to oppose the NVU's fascist views in #Amsterdam on September 22. [URL]"

In [293]:
test_tweet_off_new_4['tweet'][362]  = "#Share theleadersmindset • • • 📍Hey Leaders!🔥 ✔️Comment 'SET' if you're geared up for these actions. - 'Determine objectives, hold them close, tackle them with zeal, rejoice in your achievements... [URL]"

In [294]:
test_tweet_off_new_4['tweet'][483] ="@USER @USER @USER Your meme seems to indicate that both Fascism and Antifa stood against liberals."

In [295]:
test_tweet_off_new_4['tweet'][313] ="@USER @USER @USER A lot of people perceive her as insincere within the Senate. If she represented me, I'd be disappointed by her actions during the SCOTUS hearings."

In [296]:
test_tweet_off_new_4['tweet'][382] = "The circumstances involving #ChristineBlaseyFord have garnered notice. Some observers claim she's experiencing undue pressure from specific factions. They highlight alleged ties between Antifa, viewed by some as an extreme wing of the Democratic Party, and purported support from George Soros. The hashtag #VoteRedToSaveAmerica is associated with these discussions."

In [297]:
test_tweet_off_new_4['tweet'][853] ="Both possess a displeasing flavor. [URL]."

In [298]:
test_tweet_off_new_4.shape

(25, 2)

In [299]:

merged_df_4 = pd.concat([test_tweet_not_new, test_tweet_off_new_4], ignore_index=True)

# Shuffle the index
test_4_df = merged_df_4.sample(frac=1).reset_index(drop=True)

In [300]:
test_4_df.shape

(125, 2)

In [301]:
clean_tweets(test_4_df)

<ipython-input-10-58d0d99cd0a4>:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, 'tweet'] = df.tweet.str.replace('\d+','') #Remove numbers
<ipython-input-10-58d0d99cd0a4>:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.loc[:, 'tweet'] = df.tweet.str.replace(punctuation, '')


In [302]:
test_4_df.update(test_4_df[['tweet']].applymap('\'{}\''.format))

In [303]:

test_4_df["class_a_code"] = encoder.fit_transform(test_4_df["class_a"])

In [304]:
X_test_4 = test_4_df['tweet']
y_test_4 = test_4_df['class_a_code']

In [305]:
encoder = LabelEncoder()
train_labels = encoder.fit_transform(y_train)

test_labels_4 = encoder.transform(y_test_4)

In [306]:
test_encodings_4 = tokenizer(list(X_test_4), truncation=True, padding=True, max_length=256)

In [307]:
test_dataset_4 = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings_4),
    test_labels_4
)).batch(16)

In [308]:
# Get the model predictions
predictions_4 = tf.argmax(model.predict(test_dataset_4)[0], axis=-1)

8/8 [==============================] - 1s 87ms/step


In [309]:
# Decode the label numbers back into original labels
y_test_inverse_4 = encoder.inverse_transform(y_test_4)
predictions_inverse_4 = encoder.inverse_transform(predictions_4.numpy())

# Generate the classification report
report_4 = classification_report(y_test_inverse_4, predictions_inverse_4,
                               target_names=encoder.classes_.astype(str),digits =4)
print(report_4)

              precision    recall  f1-score   support

           0     0.8224    0.8800    0.8502       100
           1     0.3333    0.2400    0.2791        25

    accuracy                         0.7520       125
   macro avg     0.5779    0.5600    0.5647       125
weighted avg     0.7246    0.7520    0.7360       125

